In [6]:
import pandas as pd

df = pd.read_csv(r"C:\Users\avani\Desktop\Tomorrow's Projects\archive\Sample - Superstore.csv", encoding='latin1')


selected_columns = ['Order ID', 'Customer ID', 'Region', 'Product Name', 'Quantity', 'Sales', 'Order Date']
df_clean = df[selected_columns]


df_clean.to_csv(r"C:\Users\avani\Desktop\Tomorrow's Projects\archive\sales_orders_clean.csv", index=False)

print("Cleaned CSV saved as sales_orders_clean.csv")



Cleaned CSV saved as sales_orders_clean.csv


In [14]:
!pip install pandasql

import pandas as pd
import pandasql as psql
import boto3
import os


bucket_name = 'cb-retail-data-pipeline'  
raw_key = 'raw/sales_orders_clean.csv'
processed_key = 'processed/sales_orders_final.csv'

# Initialize S3 client
s3 = boto3.client('s3')

# Step 1: Download RAW file from S3
local_raw_file = 'sales_orders_clean.csv'
s3.download_file(bucket_name, raw_key, local_raw_file)
print("Downloaded RAW file from S3.")

# Step 2: Load data into Pandas
df = pd.read_csv(local_raw_file, encoding='latin1')

# Step 3: Basic Cleaning
df = df.dropna()
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df = df.dropna(subset=['Order Date'])
df['Total_Sales'] = df['Quantity'] * df['Sales']
df['Month'] = df['Order Date'].dt.to_period('M').astype(str)

# Step 4: SQL Transformations using pandasql
# Total Sales by Region
query_region = """
SELECT Region, SUM(Total_Sales) AS Total_Region_Sales
FROM df
GROUP BY Region
ORDER BY Total_Region_Sales DESC;
"""
df_region_sales = psql.sqldf(query_region, locals())

# Top 5 Products by Revenue
query_top_products = """
SELECT [Product Name], SUM(Total_Sales) AS Product_Revenue
FROM df
GROUP BY [Product Name]
ORDER BY Product_Revenue DESC
LIMIT 5;
"""
df_top_products = psql.sqldf(query_top_products, locals())

# Monthly Sales Trend
query_monthly = """
SELECT Month, SUM(Total_Sales) AS Monthly_Revenue
FROM df
GROUP BY Month
ORDER BY Month;
"""
df_monthly_sales = psql.sqldf(query_monthly, locals())

# Step 5: Save Processed Outputs Locally
local_processed_file = 'sales_orders_final.csv'
df.to_csv(local_processed_file, index=False)

# (Optional) Save summaries locally for GitHub upload
df_region_sales.to_csv('region_sales_summary.csv', index=False)
df_top_products.to_csv('top5_products_summary.csv', index=False)
df_monthly_sales.to_csv('monthly_sales_summary.csv', index=False)

print("Local Processed files saved.")

# Step 6: Upload the Processed File to S3
s3.upload_file(local_processed_file, bucket_name, processed_key)
print(f"Processed file uploaded to S3 at {bucket_name}/{processed_key}")




Defaulting to user installation because normal site-packages is not writeable
✅ Downloaded RAW file from S3.
Local Processed files saved.
Processed file uploaded to S3 at cb-retail-data-pipeline/processed/sales_orders_final.csv


Defaulting to user installation because normal site-packages is not writeable
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=2fa107fa083effa75f42012e44a5ef53b4de998622748735746f2cbb8bfa2b02
  Stored in directory: c:\users\avani\appdata\local\pip\cache\wheels\63\e8\ec\75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found